In [4]:
# MOUNT G_DRIVE
from pathlib import Path
from IPython import get_ipython
on_colab = 'google.colab' in str(get_ipython())

if on_colab:
  from google.colab import drive
  drive.mount("/content/gdrive")

PATH =  "/content/gdrive/My Drive/DeepLearning/MODELS/" if on_colab else "./"
PATH_DATASET = "/content/gdrive/My Drive/DeepLearning/DATASETS/" if on_colab else "./"

# READ DATASETS
import pandas as pd
from torch.utils.data import Dataset
import numpy as np

# FILE THAT CONTAINS ADVERSARIAL ATTACK RESULTS
FILE_NAME = "LSTM_YahooAnswers_ADV_EXAMPLES.csv" #"LSTM_AG_NEWS_ADV_EXAMPLES_5k.csv" #"CNN_IMDB_ADV_EXAMPLES.csv"

# FILE THAT CONTAINS THE ORIGINAL TRAINING SET
DIR_TRAIN = 'YahooAnswers' #"IMDB" #"AG_NEWS"
ORIG_TRAIN_NAME = "/train.csv"

# READ CSVS
df = pd.read_csv(PATH_DATASET+FILE_NAME)
if DIR_TRAIN == "AG_NEWS":
    df_orig = pd.read_csv(PATH_DATASET+DIR_TRAIN+ORIG_TRAIN_NAME, header=None)

elif DIR_TRAIN == "IMDB":
    df_orig = pd.read_csv(PATH_DATASET+DIR_TRAIN+ORIG_TRAIN_NAME)

elif DIR_TRAIN == "YahooAnswers":
    df_orig = pd.read_csv(PATH_DATASET+DIR_TRAIN+ORIG_TRAIN_NAME, header=None)


# GET LABEL AND ADVERSARIAL EXAMPLE (Successful ones)
df_modified = df[df["result_type"] == "Successful"][["perturbed_text", "ground_truth_output"]].reset_index(drop=True)

# CONVERT TO INTEGER
df_modified['ground_truth_output'] = df_modified['ground_truth_output'].apply(np.int64)

# CHANGE COLUMN NAMES OF ADVERSARIAL SAMPLES
df_modified = df_modified.rename(columns={"perturbed_text":"txt", "ground_truth_output":"label"})

# AG_NEWS HAS 2 COLUMNS CONTAINING TEXT MERGE THEM
if DIR_TRAIN == "AG_NEWS":
    df_orig["txt"] = df_orig[1] + " " + df_orig[2]
    df_orig_modified = df_orig[["txt", 0]]

    # CONVERT THE LABELS
    df_orig_modified[0] = df_orig_modified[0]-1

    # CHANGE THE COLUMN NAMES SO THAT THEY MATCH
    df_orig_modified = df_orig_modified.rename(columns = {0:"label"})

    # CONCAT THE ADV EXAMPLES AND ORIGINAL SAMPLES
    df_concat = pd.concat([df_orig_modified, df_modified]).reset_index(drop=True)

    # SAVE THE RESULT
    df_concat.to_csv(PATH_DATASET+DIR_TRAIN+"_ADV.csv", index=False)

elif DIR_TRAIN == "IMDB":

    # CONVERT POS/NEG TO 1,0
    df_orig["sentiment"].replace({"neg":0, "pos":1}, inplace=True)

    # CHANGE THE COLUMN NAMES SO THAT THEY MATCH
    df_orig = df_orig.rename(columns={"text":"txt", "sentiment":"label"})

    # CONCAT THE ADV EXAMPLES AND ORIGINAL SAMPLES
    df_concat = pd.concat([df_orig, df_modified]).reset_index(drop=True)

    # SAVE THE RESULT
    df_concat.to_csv(PATH_DATASET+DIR_TRAIN+"_ADV.csv", index=False)

elif DIR_TRAIN == "YahooAnswers":

    # SUBTRACT 1 FROM THE LABELS
    df_orig[0] = df_orig[0] - 1

    # CONVERT NAN FIELDS TO EMPTY STRING
    df_orig.fillna("", inplace=True)

    # MAKE ONE FIELD NAMED TXT
    df_orig["txt"] = df_orig[1] + " " + df_orig[2] + " " + df_orig[3]
    df_orig_modified = df_orig[["txt", 0]]

    # CHANGE THE COLUMN NAMES SO THAT THEY MATCH
    df_orig_modified = df_orig_modified.rename(columns = {0:"label"})

    # CONCAT THE ADV EXAMPLES AND ORIGINAL SAMPLES
    df_concat = pd.concat([df_orig_modified, df_modified]).reset_index(drop=True)

    # SAVE THE RESULT
    df_concat.to_csv(PATH_DATASET+DIR_TRAIN+"_ADV.csv", index=False)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# CHECK IF THE TRAIN EXAMPLE FORMED FITS THE TEMPLATE
# SCHEMA: "TXT", "LABEL" WITH ADVERSARIAL EXAMPLES AT THE END

df_result = pd.read_csv(PATH_DATASET+DIR_TRAIN+"_ADV.csv")
df_result